## Lab 5,6

### Objectives
1. Implement an SQL model in Python for smartphones.
2. Convert it to the Movies/Genres model we worked on in class.
3. Do some code changes.
4. Draw Mermaid diagrams (docs [here](https://mermaid.js.org/syntax/entityRelationshipDiagram.html)).

### Setup
First, we need to setup an SQLite connection.

In [58]:
import sqlite3

con = sqlite3.connect("movies.db")
cur = con.cursor()


In [59]:
import sqlite3
from enum import Enum

con = sqlite3.connect("movies.db")
cur = con.cursor()

class Country(Enum):
    USA = 'USA'
    UK = 'UK'
    INDIA = 'India'

    @classmethod
    def valueTuples(cls):
        """Get a list of all values in an enum.
        Note that we wrap values in single-element tuples (el,)
        so that to feed it into 'INSERT' statement.
        """
        return [(e.value,) for e in cls]

    @classmethod
    def createTable(cls):
        """This function does two things. First, creates a l_countries table.
        Second, insert predefined values into the table. This is where we use
        the valueTuples method from above.
        Note the con.commit() call - it's necessary whenever we make changes to a table
        """ 
        cur.execute('CREATE TABLE IF NOT EXISTS l_countries (country TEXT PRIMARY KEY)')
        cur.executemany('INSERT INTO l_countries VALUES(?)', cls.valueTuples())
        con.commit()

    @classmethod
    def removeTable(cls):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS l_countries')
        con.commit()
        
    @classmethod
    def getAll(cls):
        """Fetch all records from l_countries table.
        Note that we convert result tuples back to strings - 
        it's a reverse operation to the one we did in valueTuples() method.
        """
        cur.execute('SELECT * FROM l_countries')
        sqlResult = cur.fetchall()
        return [cls(p[0]) for p in sqlResult]


### Phone platforms
Let's create a Python class for phone platforms. Each phone can have a single platform (iOS/Android).

Please read code comments for all member functions, they are important for understanding how it all works.

In [60]:
Country.removeTable()
Country.createTable()
countries = Country.getAll()
print(countries)


[<Country.USA: 'USA'>, <Country.UK: 'UK'>, <Country.INDIA: 'India'>]


Let's test it. Create `l_platforms` table:

Check if platform values have been inserted:

### Markets class
Now let's create a `Market` class. It will represent list of markets the phone is sold in, like `UA` (Ukraine), `US` (United States), `DE` (Germany), `BR` (Brazil), `IN` (India).

In [62]:
class Genre(Enum):
    ACTION = 'Action'
    COMEDY = 'Comedy'
    DRAMA = 'Drama'
    HORROR = 'Horror'
    ROMANCE = 'Romance'

    @classmethod
    def valueTuples(cls):
        """Get a list of all values in an enum.
        Note that we wrap values in single-element tuples (el,)
        so that to feed it into 'INSERT' statement.
        """
        return [(e.value,) for e in cls]

    @classmethod
    def createTable(cls):
        """This function does two things. First, creates a l_genres table.
        Second, insert predefined values into the table. This is where we use
        the valueTuples method from above.
        Note the con.commit() call - it's necessary whenever we make changes to a table
        """ 
        cur.execute('CREATE TABLE IF NOT EXISTS l_genres (genre TEXT PRIMARY KEY)')
        cur.executemany('INSERT INTO l_genres VALUES(?)', cls.valueTuples())
        con.commit()

    @classmethod
    def removeTable(cls):
        """Method for dropping a table. Useful for debugging.
        """
        cur.execute('DROP TABLE IF EXISTS l_genres')
        con.commit()
        
    @classmethod
    def getAll(cls):
        """Fetch all records from l_genres table.
        Note that we convert result tuples back to strings - 
        it's a reverse operation to the one we did in valueTuples() method.
        """
        cur.execute('SELECT * FROM l_genres')
        sqlResult = cur.fetchall()
        return [cls(p[0]) for p in sqlResult]


It's very similar to the `Platform` enum above.

Let's verify that it works properly.

In [63]:
Genre.removeTable()
Genre.createTable()
genres = Genre.getAll()
print(genres)


[<Genre.ACTION: 'Action'>, <Genre.COMEDY: 'Comedy'>, <Genre.DRAMA: 'Drama'>, <Genre.HORROR: 'Horror'>, <Genre.ROMANCE: 'Romance'>]


Note that there is a 1-to-many mapping between `Phone`s and `Market`s - one phone can be sold in many markets. Therefore, we need to create an extra link table.

In [64]:
def createCountryMapping():
    cur.execute('''CREATE TABLE IF NOT EXISTS MOVIE_COUNTRY (
                    movie TEXT,
                    country TEXT,
                    PRIMARY KEY(movie, country),
                    FOREIGN KEY(movie) REFERENCES MOVIES(TITLE),
                    FOREIGN KEY(country) REFERENCES l_countries(country))''')
    con.commit()


We don't yet have a `phones` table, but that's fine. Let' run it.

In [65]:
createCountryMapping

<function __main__.createCountryMapping()>

### Phone class
Now, having all pieces in place, let's proceed to the `Phone` class. It will contain `name`, `manufacturer`, `platform`, `year`, and `rating` fields.

In [77]:
def getGenresFromString(string):
    """This is a helper method that
    constructs a set out of a comma-separated string
    """
    return {Genre(e) for e in string.split(',')}

class Movie:
    def __init__(self, title, director, genre, year, rating, countries):
        self.title = title
        self.director = director
        self.genre = genre
        self.year = year
        self.rating = rating
        self.countries = countries
    
    @classmethod
    def createTable(cls):
        """There is 1-to-1 mapping between movies and genres,
        therefore we add a FOREIGN KEY to movies table
        """
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        title TEXT PRIMARY KEY,
                        director TEXT,
                        genre TEXT,
                        year INTEGER,
                        rating REAL,
                        FOREIGN KEY(genre) REFERENCES l_genres(genre))''')
        con.commit()

    def insert(self):
        """Insert or update a movie record."""
        cur.execute('''SELECT * FROM movies WHERE title=?''', (self.title,))
        existing_record = cur.fetchone()
        if existing_record:
            self.update()
        else:
            cur.execute('''INSERT INTO movies VALUES(?,?,?,?, ?)''', (self.title, self.director, self.genre.value, self.year, self.rating))
            cur.executemany('''INSERT INTO movie_country VALUES(?,?)''', [(self.title, c.value) for c in self.countries])
            con.commit()

    def update(self):
        """Update a movie record."""
        cur.execute('''UPDATE movies SET director=?, genre=?, year=?, rating=? WHERE title=?''', (self.director, self.genre.value, self.year, self.rating, self.title))
        cur.execute('''DELETE FROM movie_country WHERE movie=?''', (self.title,))
        cur.executemany('''INSERT INTO movie_country VALUES(?,?)''', [(self.title, c.value) for c in self.countries])
        con.commit()

    @classmethod
    def get(cls, title):
        """This query uses a LEFT JOIN and an aggregate function GROUP_CONCAT
        to fetch countries for a movie.
        """
        cur.execute('''SELECT m.director, m.genre, m.year, m.rating, GROUP_CONCAT(mc.country, ',') FROM movies m
                    LEFT JOIN movie_country mc ON m.title = mc.movie
                    WHERE m.title=? GROUP BY m.title''', (title,))
        res = cur.fetchone()
        return Movie(title, res[0], Genre(res[1]), res[2], res[3], [Country(c) for c in res[4].split(',')])

    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.title, self.director, self.genre, self.year, self.rating, self.countries)


In [78]:
Movie.createTable()

In [79]:

movie1 = Movie('Inception', 'Christopher Nolan', Genre.ACTION, 2010, 0, [Country.USA])
movie2 = Movie('Interstellar', 'Christopher Nolan', Genre.DRAMA, 2014, 0, [Country.USA, Country.UK])
movie3 = Movie('The Shawshank Redemption', 'Frank Darabont', Genre.DRAMA, 1994, 0, [Country.USA])

movie1.insert()
movie2.insert()
movie3.insert()


In [80]:
print(Movie.get('Interstellar'))

(Interstellar, Christopher Nolan, Genre.DRAMA, 2014, 0.0, [<Country.UK: 'UK'>, <Country.USA: 'USA'>])


Below is a helper function to be used in exercise 3.

In [57]:
def getMarketsFromString(str):
    """This is a helper method that
    constructs a set out of a comma-separated string
    """
    return {Market(e) for e in str.split(',')}

### Exercises

1. Convert the code above to use Movies/Genres model. So, instead of `Phone`, there will be `Movie`, and instead of `Market`, there will be `Genre`. Replace `Platform` with `Country`. So:
   - `Platform` -> `Country`
   - `Market` -> `Genre`
   - `Phone` -> `Movie`
2. Modify `Movie.update` to update all fields, not only rating.
3. Use `getMarketsFromString` defined above when fetching movie genres in `get` method. It will help convert a comma-separated string to a Python set.

## Lab 7,8

### Phone manufacturers

Great. Now let's add a third ingredient to the mix - a `Manufacturer` class. With this, we will specify some data about the manufacturer of the phone, like `name`, `website`, `year` (when the company was founded), `hq` (meaning headquarters - location of main office).

In [81]:
class Director:
    def __init__(self, name, birth_year, country):
        self.name = name
        self.birth_year = birth_year
        self.country = country
    
    @classmethod
    def createTable(cls):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        birth_year INTEGER,
                        country TEXT)''')
        con.commit()

    @classmethod
    def removeTable(cls):
        cur.execute('DROP TABLE IF EXISTS directors')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (self.name, self.birth_year, self.country))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET birth_year=?, country=? WHERE name=?''', (self.birth_year, self.country, self.name))
        con.commit() 

    @classmethod
    def get(cls, name):
        cur.execute('SELECT birth_year, country FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        return Director(name, res[0], res[1])

    @classmethod
    def getAll(cls):
        cur.execute('SELECT name, birth_year, country FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], el[2]) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.birth_year, self.country)

Now we check how it works.

In [82]:
Director.removeTable()
Director.createTable()

In [83]:

d1 = Director('Christopher Nolan', 1970, 'UK')
d2 = Director('Frank Darabont', 1959, 'USA')
d3 = Director('Steven Spielberg', 1946, 'USA')

d1.insert()
d2.insert()
d3.insert()


In [84]:
directorFromDb = Director.get('Steven Spielberg')
print(directorFromDb)


(Steven Spielberg, 1946, USA)


In [85]:
[print(e) for e in Director.getAll()]


(Christopher Nolan, 1970, UK)
(Frank Darabont, 1959, USA)
(Steven Spielberg, 1946, USA)


[None, None, None]

In [ ]:
erDiagram
    movies ||--o{ movie_country : "1" -- "N"
    directors ||--o{ movies : "1" -- "N"
    genres ||--o{ movies : "1" -- "N"
    movies {
        title VARCHAR(255) : PK
        director VARCHAR(255) : FK
        genre VARCHAR(255) : FK
        year INTEGER
        rating REAL
    }
    movie_country {
        movie VARCHAR(255) : FK
        country VARCHAR(255) : FK
    }
    directors {
        name VARCHAR(255) : PK
        birth_year INTEGER
        country VARCHAR(255)
    }
    genres {
        genre VARCHAR(255) : PK
    }


### Exercises
1. Convert `Phone.Manufacturer` to `Movie.Director`, having these fields:
   - `name`
   - `birth_year`
   - `country`
2. Link `Director` table to `Movie`, so that director data are fetched in `Movie.get` (not just name). Hint: use `LEFT JOIN`, similarly to market fetching in the code above.
3. Draw a Mermaid.js diagram of the resulting SQL data model (for `Movie`s, `Country`s, `Director`s, and `Genre`s)

 ## References
1. Python SQLite tutorial: https://docs.python.org/3/library/sqlite3.html#sqlite3-tutorial.
2. SQLite foreign keys: https://www.sqlite.org/foreignkeys.html.
3. SQLite aggregate functions: https://www.sqlite.org/lang_aggfunc.html.
4. Mermaid.js intro: https://mermaid.js.org/intro/.
5. Mermaid.js entity-relationship diagram docs: https://mermaid.js.org/syntax/entityRelationshipDiagram.html.